# Convergence of the optical spectra
[Link to tutorial](https://www.octopus-code.org/documentation/13/tutorial/response/convergence_of_the_optical_spectra/)

In the previous [Optical Spectra from TD](1_Optical_spectra_from_time_propagation.ipynb) we have seen how to obtain the absorption spectrum of a molecule. Here we will perform a convergence study of the spectrum with respect to the grid parameters using again the methane molecule as an example.


In [ ]:
import os
import subprocess
from string import Template

import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
!mkdir -p 2_Convergence_of_optical_spectra

In [ ]:
cd 2_Convergence_of_optical_spectra

## Convergence with the spacing

In the [Total energy convergence tutorial](../1-basics/3-Total_energy_convergence.ipynb) we found out that the total energy of the methane molecule was converged to within 0.1 eV for a spacing of 0.18 Å and a radius of 3.5 Å. We will therefore use these values as a starting point for our convergence study. Like for the total energy, the only way to check if the absorption spectrum is converged with respect to the spacing is by repeating a series of calculations, with identical input files except for the grid spacing. The main difference in this case is that each calculation for a given spacing requires several steps:

- Ground-state calculation,
- Three time-dependent propagations with perturbations along ''x'', ''y'' and ''z'',
- Calculation of the spectrum using the `oct-propagation_spectrum` utility.

As we have seen before, the $T_d$ symmetry of methane means that the spectrum is identical for all directions, so in this case we only need to perform the convergence study with a perturbation along the ''x'' direction.

All these calculations can be done by hand, but we would rather use a script for this boring an repetitive task. We will then need the following file.

#### inp
This file is the same as the one used in the previous [Optical Spectra from TD](1_Optical_spectra_from_time_propagation.ipynb):


In [ ]:
inp_template_methane_spacing = Template(
    """
    stdout = 'stdout_${Calculation_value}_${Spacing_value}.txt'
    stderr = 'stderr_${Calculation_value}_${Spacing_value}.txt'

    CalculationMode = ${Calculation_value}
    FromScratch = yes
    UnitsOutput = eV_angstrom

    Radius = 3.5*angstrom
    Spacing = ${Spacing_value}*angstrom

    CH = 1.097*angstrom
    %Coordinates
    "C" |           0 |          0 |           0
    "H" |  CH/sqrt(3) | CH/sqrt(3) |  CH/sqrt(3)
    "H" | -CH/sqrt(3) |-CH/sqrt(3) |  CH/sqrt(3)
    "H" |  CH/sqrt(3) |-CH/sqrt(3) | -CH/sqrt(3)
    "H" | -CH/sqrt(3) | CH/sqrt(3) | -CH/sqrt(3)
    %

    TDPropagator = aetrs
    TDTimeStep = 0.0023/eV
    TDMaxSteps = 4350  # ~ 10.0/TDTimeStep

    TDDeltaStrength = 0.01/angstrom
    TDPolarizationDirection = 1   
    """
)

This script performs the three steps mentioned above for a given list of spacings. You will notice that we are not including smaller spacings than 0.18 Å. This is because we know from previous experience that the optical spectrum is well converged for spacings larger than the one required to converge the total energy.

This will take a few minutes.

In [ ]:
spacing_list = [0.26, 0.24, 0.22, 0.20, 0.18]

for spacing in spacing_list:
    print(f"Running octopus for spacing: {spacing}")

    os.makedirs(f"methane/variable_spacing/run_{spacing}")

    with open(f"methane/variable_spacing/run_{spacing}/inp", "w") as inp_file:
        inp_file.write(
            inp_template_methane_spacing.substitute(
                Spacing_value=spacing, Calculation_value="gs"
            )
        )
    subprocess.run("octopus", shell=True, cwd=f"methane/variable_spacing/run_{spacing}")

    with open(f"methane/variable_spacing/run_{spacing}/inp", "w") as inp_file:
        inp_file.write(
            inp_template_methane_spacing.substitute(
                Spacing_value=spacing, Calculation_value="td"
            )
        )
    subprocess.run(
        "octopus && oct-propagation_spectrum",
        shell=True,
        cwd=f"methane/variable_spacing/run_{spacing}",
    )

Once the script is finished running, we need to compare the spectra for the different spacings. You can see on the right how this plot should look like. To make the plot easier to read, we have restricted the energy range and omitted some of the spacings. From the plot it is clear that a spacing of 0.24 Å is sufficient to have the position of the peaks converged to within 0.1 eV.

In [ ]:
fig, ax = plt.subplots()
ax.set_title("Absorption spectrum of methane for different spacings")
ax.set_xlabel("Energy (eV)")
ax.set_ylabel("Strength function (1/eV)")

for spacing in spacing_list:
    df = pd.read_csv(
        f"methane/variable_spacing/run_{spacing}/cross_section_vector",
        delimiter="     ",
        usecols=[0, 4],
        names=["Energy (eV)", "StrengthFunction"],
        skiprows=26,
        engine="python",
    )
    df.plot(
        x="Energy (eV)", y="StrengthFunction", ax=ax, label=f"{spacing}", xlim=[9, 15]
    )

## Convergence with the radius

We will now study how the spectrum changes with the radius. We will proceed as for the spacing, by performing a series of calculations with identical input files except for the radius. For this we will use the following file.

#### inp
This file is the same as the prevous one, with the exception of the spacing and the time-step. As we have just seen, a spacing of 0.24 Å is sufficient, which in turns allows us to use a larger time-step.


In [ ]:
inp_template_methane_radius = Template(
    """
    stdout = 'stdout_${Calculation_value}_${Radius_value}.txt'
    stderr = 'stderr_${Calculation_value}_${Radius_value}.txt'

    CalculationMode = ${Calculation_value}
    FromScratch = yes
    UnitsOutput = eV_angstrom

    Radius = $Radius_value*angstrom
    Spacing = 0.24*angstrom

    CH = 1.097*angstrom
    %Coordinates
    "C" |           0 |          0 |           0
    "H" |  CH/sqrt(3) | CH/sqrt(3) |  CH/sqrt(3)
    "H" | -CH/sqrt(3) |-CH/sqrt(3) |  CH/sqrt(3)
    "H" |  CH/sqrt(3) |-CH/sqrt(3) | -CH/sqrt(3)
    "H" | -CH/sqrt(3) | CH/sqrt(3) | -CH/sqrt(3)
    %

    TDPropagator = aetrs

    TDTimeStep = 0.004/eV
    TDMaxSteps = 2500  # ~ 10.0/TDTimeStep

    TDDeltaStrength = 0.01/angstrom
    TDPolarizationDirection = 1
    """
)

The time requiered to create all runs are in the order of 11 minutes.
This example provides an alternative to the use of subprocess.

In [ ]:
list_of_radii = [3.5, 4.5, 5.5, 6.5, 7.5]

for radius in list_of_radii:
    print(f"Running octopus for radius: {radius}")

    os.makedirs(f"methane/variable_radius/run_{radius}")

    run_path = "methane/variable_radius/run_" + str(radius)
    inp_file_path = run_path + "/inp"

    with open(inp_file_path, "w") as inp_file:
        inp_file.write(
            inp_template_methane_radius.substitute(
                Radius_value=radius, Calculation_value="gs"
            )
        )
    !cd {run_path} && octopus

    with open(inp_file_path, "w") as inp_file:
        inp_file.write(
            inp_template_methane_radius.substitute(
                Radius_value=radius, Calculation_value="td"
            )
        )
    !cd {run_path} && octopus && oct-propagation_spectrum

Now plot the spectra from the `cross_section_vector` files. We see that in this case the changes are quite dramatic. To get the first peak close to 10 eV converged within 0.1 eV a radius of 6.5 Å is necessary. The converged transition energy of 9.2 eV agrees quite well with the experimental value of 9.6 eV and with the TDDFT value of 9.25 eV obtained by other authors.[$^1$](#first_Reference)

In [ ]:
fig, ax = plt.subplots()
ax.set_title("Absorption spectrum of methane for different radii")
ax.set_xlabel("Energy (eV)")
ax.set_ylabel("Strength function (1/eV)")

for radius in list_of_radii:
    df = pd.read_csv(
        f"methane/variable_radius/run_{radius}/cross_section_vector",
        delimiter="     ",
        usecols=[0, 4],
        names=["Energy (eV)", "StrengthFunction"],
        skiprows=26,
        engine="python",
    )
    df.plot(
        x="Energy (eV)", y="StrengthFunction", ax=ax, label=f"{radius}", xlim=[7, 15]
    )


What about the peaks at higher energies? Since we are running in a box with zero boundary conditions, we will also have the box states in the spectrum. The energy of those states will change with the inverse of the size of the box and the corresponding peaks will keep shifting to lower energies. However, as one is usually only interested in bound-bound transitions in the optical or near-ultraviolet regimes, we will stop our convergence study here. As an exercise you might want to converge the next experimentally observed transition. Just keep in mind that calculations with larger boxes might take quite some time to run.

[Go to *3_Optical_spectra_from_casida.ipynb*](3_Optical_spectra_from_casida.ipynb)

## References 

1. N. N. Matsuzawa, A. Ishitani, D. A. Dixon, and T. Uda, Time-Dependent Density Functional Theory Calculations of Photoabsorption Spectra in the Vacuum Ultraviolet Region, [J. Phys. Chem. A](https://doi.org/10.1021/jp003937v) 105 4953–4962 (2001);
<span id="first_reference"></span>